In [1]:
import ee
import geemap
from gee_0_utils import *
initialize()

config = ProjectConfig()
roi = config.roi
data_folder = config.data_folder
last_year = config.last_year


- Total secondary forest area per 1km2
- Total pasture area per 1km2

In [2]:
lulc = (ee.Image("projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1")
            .select([f"classification_{year}" for year in config.range_1985_2020])
            .byte()
            .rename([str(year) for year in config.range_1985_2020]))

pastureland = lulc.select("2020").eq(15).unmask(0).rename("pastureland")

# Aggregate the high-resolution pixels into the 1km grid
def reproject_export_image(image, name):
    image_area = image.reduceResolution(
        reducer = ee.Reducer.mean(),
        maxPixels = 1024,
        bestEffort = True # Use all pixels that can fit in the larger pixel
    ).reproject(
        crs = 'EPSG:4326',
        scale = 1000
    ).rename(name)

    export_image(image_area, name, region = roi, scale = 1000)


# reproject_export_image(pastureland, "pasture_area")
# reproject_export_image(age.gt(0).unmask(0), "secondary_area")

- 1 secondary forest pixel per 1km2
- 1 pasture pixel per 1km2

In [3]:

def create_grid(image, region_name = "amazon", cell_size = 10000, file_name = None):
    
    config = ProjectConfig()
    biomes = ee.Image(f"{config.data_folder}/categorical").select("biome")
    
    if region_name == "amazon":
        biome_index = 1
    elif region_name == "atlantic":
        biome_index = 4
    
    biomes = biomes.eq(biome_index).selfMask()

    pixels_to_sample = biomes.reduceResolution(
        ee.Reducer.first(), maxPixels=65536
        ).reproject(
        crs = image.projection().getInfo()['crs'],
        crsTransform = image.projection().getInfo()['transform']
        ).updateMask(image)
    
    image_scale = round(pixels_to_sample.projection().nominalScale().getInfo())
    closest_multiple = round(cell_size / image_scale) * image_scale

    # First, sample locations based only on the age band
    grid = geemap.create_grid(pixels_to_sample.geometry(), closest_multiple, pixels_to_sample.projection())

    # Function to sample one point per valid cell
    def sample_cell(cell):
        sampled_fc = pixels_to_sample.stratifiedSample(
            numPoints = 1,
            classBand = 'biome',
            region = cell.geometry(),
            scale = pixels_to_sample.projection().nominalScale(),
            geometries = True,
            dropNulls = True,
            tileScale = 1
        )

        # Only return a feature if we found one
        return ee.Feature(ee.Algorithms.If(
            sampled_fc.size().gt(0),
            sampled_fc.first(),
            # Return a placeholder that we can filter out later
            ee.Feature(ee.Geometry.Point([0, 0])).set('is_null', True)
        ))

    samples = grid.map(sample_cell)

    # Filter out placeholder features before exporting
    samples = samples.filter(ee.Filter.notEquals('is_null', True))

    if file_name is None:
        return samples
    else:
        export_name = f"grid_{cell_size//1000}k_{region_name}_{file_name}"

        export_task = ee.batch.Export.table.toAsset(
            collection = samples,
            description = export_name,
            assetId = f"{config.data_folder}/{export_name}"
        )
        export_task.start()



In [4]:
mature_biomass_10k = ee.Image(f"{data_folder}/mature_biomass_10k")
age = ee.Image(f"{data_folder}/mapbiomas_2020")

In [ ]:
# create_grid(age.gt(0).unmask(0), region_name = "amazon", cell_size = 1000, file_name = "secondary")
# create_grid(age.gt(0).unmask(0), region_name = "amazon", cell_size = 10000, file_name = "secondary")
# create_grid(pastureland, region_name = "amazon", cell_size = 10000, file_name = "pastureland")

# make one grid with all pixels that are NOT mature forests for nearest mature forest estimates
# non_mature_forests = mature_biomass_10k.eq(0).unmask(1).selfMask().rename("non_mature_forests")
# create_grid(non_mature_forests, "amazon", cell_size = 10000, file_name = "non_mature_forests")

In [ ]:
# make one unified grid for all pastureland and secondary forests
grid_secondary = ee.FeatureCollection(f"{data_folder}/grid_1k_amazon_secondary")
grid_pastureland = ee.FeatureCollection(f"{data_folder}/grid_1k_amazon_pastureland")

# merge both grids
grid_1k_amazon_non_mature_forests = grid_secondary.merge(grid_pastureland)

task = ee.batch.Export.table.toAsset(
    collection = grid_1k_amazon_non_mature_forests,
    description = "grid_1k_amazon_non_mature_forests",
    assetId = "projects/amazon-forest-regrowth/assets/grid_1k_amazon_non_mature_forests",
)
# task.start()
